In [16]:
import pandas as pd


In [17]:
file = '/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/Jan19.txt'
out='/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/criteria_above_discard_redundant_clinsig.txt'

In [19]:
with open(file,'r') as f:
    with open(out,'w') as f_out:
        line=f.readline()
        f_out.write('clinvar_id;review_status;clinical_sig;uniprot_kb;variant_type;hgvs_p;missense\n')
        while True:
            line=f.readline()

            if line.count(';')>6:
                discard_num=line.count(';')-6+3
                line_new=';'.join(line.split(';')[:3]+line.split(';')[discard_num:])
                print(set(line.split(';'))-set(line_new.split(';')),' is discarded')
                f_out.write(line_new)
            else:
                f_out.write(line)
            if not line:break

{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' association', ' risk factor'}  is discarded
{' association'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' association'}  is discarded
{' risk factor', ' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is 

# Inspect data

In [20]:
data=pd.read_csv(out,delimiter=';')
data.head()

,clinvar_id,review_status,clinical_sig,uniprot_kb,variant_type,hgvs_p,missense
0,SCV002232601,"criteria provided, single submitter",Pathogenic,P04181#VAR_000578,Variation,P04181:p.Arg271Lys,missense variant
1,SCV002291827,"criteria provided, single submitter",Uncertain significance,P04181#VAR_015648,Variation,P04181:p.Gln90Glu,missense variant
2,SCV001445903,"criteria provided, single submitter",Likely pathogenic,P12955#VAR_011615,Variation,P12955:p.Gly278Asp,missense variant
3,SCV001449002,"criteria provided, single submitter",Pathogenic/Likely pathogenic,P12955#VAR_004405,Variation,P12955:p.Gly448Arg,missense variant
4,SCV000935565,"criteria provided, single submitter",Likely pathogenic,P21359#VAR_002663,Variation,P21359:p.Leu1953Pro,missense variant


In [22]:
data.clinical_sig.unique()

array(['Pathogenic', 'Uncertain significance', 'Likely pathogenic',
       'Pathogenic/Likely pathogenic', 'Benign', 'Likely benign',
       'Conflicting interpretations of pathogenicity',
       'Benign/Likely benign', 'drug response', 'other',
       'Likely risk allele', 'risk factor',
       'Pathogenic/Likely pathogenic/Established risk allele',
       'Established risk allele', 'Pathogenic/Pathogenic, low penetrance',
       'association'], dtype=object)

In [34]:
data.variant_type.unique()

array(['Variation', 'single nucleotide variant', 'copy number gain'],
      dtype=object)

# Process clinical significance

['Pathogenic', 'Uncertain significance', 'Likely pathogenic',
       'Pathogenic/Likely pathogenic', 'Benign', 'Likely benign',
       'Conflicting interpretations of pathogenicity',
       'Benign/Likely benign', 'drug response', 'other',
       'Likely risk allele', 'risk factor',
       'Pathogenic/Likely pathogenic/Established risk allele',
       'Established risk allele', 'Pathogenic/Pathogenic, low penetrance',
       'association']

In [29]:
def if_positive_or_negative(string_list):
    label=[]
    for string in string_list:
        if string in ['Pathogenic', 
                    'Pathogenic/Likely pathogenic', 
                    'probable-pathogenic',
                    'Likely pathogenic', 
                    'pathologic', 
                    'pathogenic',
                    'likely pathogenic',
                    'Pathogenic/Likely pathogenic/Established risk allele',
                    'likely pathogenic - adrenal pheochromocytoma',
                    'Pathogenic/Pathogenic, low penetrance',
                    'Pathogenic, low penetrance']:
            label.append(1)
        elif string in ['Benign',
                    'Likely benign',
                    'Likely Benign',
                    'Benign/Likely benign',
                    'non-pathogenic', 
                    'benign', 'probable-non-pathogenic', 'Likely Benign', 'probably not pathogenic',
        ]:
            label.append(-1)

        else: label.append(0)
    return label

In [30]:
data['label']=if_positive_or_negative(data['clinical_sig'])

## Generate Positive Clinvar File and Negative Clinvar File

In [32]:
data[data['label']==1].to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/clinvar_pos.csv',sep=';')
data[data['label']==-1].to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/clinvar_neg.csv',sep=';')

In [35]:
data[data['label']==-1]

,clinvar_id,review_status,clinical_sig,uniprot_kb,variant_type,hgvs_p,missense,label
19,SCV000361195,"criteria provided, single submitter",Benign,P0C7Q2#VAR_044331,Variation,P0C7Q2:p.Ala69Ser,missense variant,-1
21,SCV001325434,"criteria provided, single submitter",Likely benign,O60393#VAR_036636,Variation,O60393:p.Arg355His,missense variant,-1
55,SCV000451797,"criteria provided, single submitter",Likely benign,Q9NRM0#VAR_065775,Variation,Q9NRM0:p.Arg198Cys,missense variant,-1
58,SCV001136513,"criteria provided, single submitter",Benign,Q9NP91#VAR_052068,Variation,Q9NP91:p.Thr199Met,missense variant,-1
79,SCV001138159,"criteria provided, single submitter",Benign,P15169#VAR_042415,Variation,P15169:p.Gly178Asp,missense variant,-1
...,...,...,...,...,...,...,...,...
32189,SCV002738545,"criteria provided, single submitter",Benign,P17813#VAR_014764,Variation,P17813:p.Asp366His,missense variant,-1
32191,SCV002612763,"criteria provided, single submitter",Benign,Q14896#VAR_074545,Variation,Q14896:p.Arg1138His,missense variant,-1
32192,SCV002613575,"criteria provided, single submitter",Likely benign,Q14896#VAR_020571,Variation,Q14896:p.Leu383Val,missense variant,-1
32196,SCV002614959,"criteria provided, single submitter",Benign,Q8TAB3#VAR_067489,Variation,Q8TAB3:p.Asn1134His,missense variant,-1


In [3]:
def generate_example(start,end):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_2021-06.xml', 'r') as f_in:
        with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/example_'+str(start)+'_'+str(end)+'.xml','w') as f_out:
            line_count=0
            while True:
                line=f_in.readline()
                line_count+=1
                if line_count>start and line_count<end:
                    f_out.write(line)
                if line_count>end:
                    break

def find_line(content):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_2021-06.xml', 'r') as f_in:
            line_count=0

            while True:
                line=f_in.readline()
                line_count+=1

                if content in line:
                    print(line_count)
                    break

In [5]:
generate_example(45667191,45669191)

In [3]:
f.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,...,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,...,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4827360,GCTGCTGGACCTGCC,G
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,no assertion criteria provided,1,-,N,"OMIM:613624.0001,ClinGen:CA210674,UniProtKB:Q9...",1,4,85342440,G,A


In [4]:
f.columns

Index(['#AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol', 'HGNC_ID',
       'ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated', 'RS# (dbSNP)',
       'nsv/esv (dbVar)', 'RCVaccession', 'PhenotypeIDS', 'PhenotypeList',
       'Origin', 'OriginSimple', 'Assembly', 'ChromosomeAccession',
       'Chromosome', 'Start', 'Stop', 'ReferenceAllele', 'AlternateAllele',
       'Cytogenetic', 'ReviewStatus', 'NumberSubmitters', 'Guidelines',
       'TestedInGTR', 'OtherIDs', 'SubmitterCategories', 'VariationID',
       'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF'],
      dtype='object')

In [9]:
f.loc[:,['#AlleleID', 'Type', 'Name','ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated','RCVaccession', 'PhenotypeIDS', 'PhenotypeList','Origin', 'OriginSimple', 'Assembly','ReviewStatus', 'NumberSubmitters', 'Guidelines' ]].head()

,#AlleleID,Type,Name,ClinicalSignificance,ClinSigSimple,LastEvaluated,RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ReviewStatus,NumberSubmitters,Guidelines
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,Pathogenic,1,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh37,"criteria provided, single submitter",2,-
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,Pathogenic,1,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh38,"criteria provided, single submitter",2,-
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),Pathogenic,1,"Jun 29, 2010",RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh37,no assertion criteria provided,1,-
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),Pathogenic,1,"Jun 29, 2010",RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh38,no assertion criteria provided,1,-
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),Uncertain significance,0,"Jun 29, 2015",RCV000000014,"MONDO:MONDO:0033005,MedGen:C4551772,OMIM:25130...",Galloway-Mowat syndrome 1,germline,germline,GRCh37,no assertion criteria provided,1,-


# Dealing with the variant summary file

In [21]:
import pandas as pd
f=pd.read_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary.txt',sep='\t')

/tmp/ipykernel_124161/2855674157.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  f=pd.read_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary.txt',sep='\t')


In [22]:
f.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,...,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,...,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4827360,GCTGCTGGACCTGCC,G
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,no assertion criteria provided,1,-,N,"OMIM:613624.0001,ClinGen:CA210674,UniProtKB:Q9...",1,4,85342440,G,A


In [66]:
def if_positive_or_negative(string_list):
    label=[]
    for string in string_list:
        if string in ['Pathogenic', 
                    'Pathogenic/Likely pathogenic', 
                    'probable-pathogenic',
                    'Likely pathogenic', 
                    'pathologic', 
                    'pathogenic',
                    'likely pathogenic',
                    'Pathogenic/Likely pathogenic/Established risk allele',
                    'likely pathogenic - adrenal pheochromocytoma',
                    'Pathogenic/Pathogenic, low penetrance',
                    'Pathogenic, low penetrance']:
            label.append(1)
        elif string in ['Benign',
                    'Likely benign',
                    'Likely Benign',
                    'Benign/Likely benign',
                    'non-pathogenic', 
                    'benign', 'probable-non-pathogenic', 'Likely Benign', 'probably not pathogenic',
        ]:
            label.append(-1)

        else: label.append(0)
    return label
f['label']=if_positive_or_negative(f['ClinicalSignificance'])
keep_cols=['#AlleleID', 'Type', 'Name','label', 'ClinicalSignificance', 'ClinSigSimple', 'ReviewStatus', 'OtherIDs', 'LastEvaluated', 'RS# (dbSNP)',
       'nsv/esv (dbVar)', 'RCVaccession']
keep_conditions=(f['Type']=='single nucleotide variant') &\
              (['no assertion' not in item for item in f['ReviewStatus'].tolist()]) &\
               (['p' in item for item in f['Name'].tolist()]) &\
               (f['label']!=0)
f_simple=f[keep_conditions][keep_cols]
f_simple.head()



,#AlleleID,Type,Name,label,ClinicalSignificance,ClinSigSimple,ReviewStatus,OtherIDs,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession
6,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),1,Pathogenic,1,"criteria provided, multiple submitters, no con...","ClinGen:CA113792,OMIM:613622.0001","Jun 28, 2022",267606829,-,RCV000000015|RCV000578659|RCV001194045
32,15058,single nucleotide variant,NM_000410.4(HFE):c.848A>C (p.Gln283Pro),1,Pathogenic,1,"criteria provided, single submitter","ClinGen:CA280949,UniProtKB:Q30201#VAR_037304,O...","Sep 12, 2022",111033563,-,RCV000000036|RCV001050090
40,15062,single nucleotide variant,NM_020779.4(WDR35):c.2590G>A (p.Ala864Thr),1,Likely pathogenic,1,"criteria provided, single submitter","UniProtKB:Q9P2L0#VAR_064582,OMIM:613602.0004,C...","May 25, 2017",267607175,-,RCV000000040|RCV000508347
48,15066,single nucleotide variant,NM_001042472.3(ABHD12):c.1054C>T (p.Arg352Ter),1,Pathogenic,1,"criteria provided, multiple submitters, no con...","ClinGen:CA113811,OMIM:613599.0004","Jan 05, 2022",267606624,-,RCV000000044|RCV001208516
54,15069,single nucleotide variant,NM_138413.4(HOGA1):c.860G>T (p.Gly287Val),1,Pathogenic/Likely pathogenic,1,"criteria provided, multiple submitters, no con...","ClinGen:CA113813,UniProtKB:Q86XE5#VAR_064036,O...","Oct 13, 2022",138207257,-,RCV000000047|RCV000798240


In [70]:
from tqdm import tqdm
import multiprocessing
num_processes = multiprocessing.cpu_count()
chunk_size = int(f_simple.shape[0]/num_processes)
global pbar
pbar = tqdm(total=len(f_simple))
f_simple['UniPort']=[0]*len(f_simple)
chunks = [f_simple.iloc[f_simple.index[i:i + chunk_size]] for i in range(0, f_simple.shape[0], chunk_size)]
def func(df):
    for idx in df.index:
        if 'UniProt' not in df.loc[idx,'OtherIDs']:
                df.loc[idx,'UniPort']=get_uniprot_from_name(f_simple.loc[idx,'Name'])
        pbar.update(1)
    return df
pool = multiprocessing.Pool(processes=num_processes)
result = pool.map(func, chunks)


  0%|          | 646/558616 [04:48<66:51:00,  2.32it/s] 

Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001191016.3


  0%|          | 1007/558616 [07:28<33:30:49,  4.62it/s]

Error Finding Refseq


  0%|          | 1032/558616 [07:38<98:10:26,  1.58it/s]

In [68]:
f_simple.index

Int64Index([      6,      32,      40,      48,      54,      60,      62,
                 64,      66,      70,
            ...
            4323230, 4323238, 4323259, 4323261, 4323285, 4323293, 4323305,
            4323310, 4323312, 4323316],
           dtype='int64', length=558616)

In [59]:
f_simple['UniProtID']=[get_uniprot_from_name(name) for name in f_simple['Name']]

Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001142446.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001142446.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001127255.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001127255.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001127255.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001127255.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001277059.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001277059.2
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001191016.3
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001191016.3
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_199451.3
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_199451.3
Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_0013

UnboundLocalError: local variable 'refseq' referenced before assignment

In [63]:
f_simple=f_simple.drop_duplicates(subset=['#AlleleID'])


### Map RefSeqID to Uniprot ID

In [1]:
import requests
import json
import re
def find_fasta_refseq(refseqID):
    url='https://rest.uniprot.org/uniprotkb/search?query=%s'%refseqID
    cnt=json.loads(requests.get(url).text)
    try:
        assert re.search('uniprot',cnt.get("results")[0].get("entryType"),re.IGNORECASE)
        return cnt["results"][0]['primaryAccession']
    except:
        print('Error found in '+url)
        return 'Error found in '+url

def get_uniprot_from_name(name):
    try:
        refseq=re.match(r'\S*?(?=[\(:])',name).group(0)
        uniprot=find_fasta_refseq(refseq)

    except:
        print('Error Finding Refseq')
        uniprot=find_fasta_refseq(refseq)
    return uniprot 

### Extract UniProt from OtherIDs    

In [5]:
import re
def extract_uniprot(string):
    try: m=re.match(r'.*UniProtKB:([0-9A-Z].*)#.*',string).group(1)
    except AttributeError: 
        print(string)
        m='0'
    return m

### Merge multiprocess files

In [32]:

def merge(dir,prefix,out_name):
    d={}
    import os
    f_list=os.listdir(dir)
    for f in f_list:
        print(f)
        proc,num=re.match(r'%s([0-9]*)_([0-9]*)_finished.csv'%prefix,f).group(1),\
        int(re.match(r'%s([0-9]*)_([0-9]*)_finished.csv'%prefix,f).group(2))
        if d.get('proc') is None or num> d.get('proc'):
            d['proc']=num
    f_out_list=[]
    for key in d.keys():
        f_out_list.append(os.path.join(dir,'%s%s_%s_finished.csv'%(prefix,proc,num)))
    print(f_out_list)
    dfs=[]
    for f in f_out_list:
        dfs.append(pd.read_csv('f',index_col='#AlleleID')
                )
    df_out=pd.concat(dfs)
    df_out.to_csv(out_name)

In [33]:
merge('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/2022-2-variant-sum-round2','2022-2-remain_proc_','merged_2022-2-round2')

2022-2-remain_proc_100218_0_finished.csv
2022-2-remain_proc_100218_100_finished.csv
2022-2-remain_proc_100218_200_finished.csv
2022-2-remain_proc_100218_300_finished.csv
2022-2-remain_proc_100218_400_finished.csv
2022-2-remain_proc_100218_500_finished.csv
2022-2-remain_proc_100218_600_finished.csv
2022-2-remain_proc_100218_700_finished.csv
2022-2-remain_proc_100218_800_finished.csv
2022-2-remain_proc_100218_900_finished.csv
2022-2-remain_proc_104680_0_finished.csv
2022-2-remain_proc_104680_100_finished.csv
2022-2-remain_proc_104680_200_finished.csv
2022-2-remain_proc_104680_300_finished.csv
2022-2-remain_proc_104680_400_finished.csv
2022-2-remain_proc_104680_500_finished.csv
2022-2-remain_proc_104680_600_finished.csv
2022-2-remain_proc_104680_700_finished.csv
2022-2-remain_proc_104680_800_finished.csv
2022-2-remain_proc_104680_900_finished.csv
2022-2-remain_proc_1095116_0_finished.csv
2022-2-remain_proc_1095116_100_finished.csv
2022-2-remain_proc_1095116_200_finished.csv
2022-2-remain_

FileNotFoundError: [Errno 2] No such file or directory: 'f'

In [46]:
g=re.match(r'\S*?(?=[\(:])','NM_000374.5:c.499G>A (p.Glu167Lys)')

In [21]:
None>1

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [19]:
print(find_fasta_refseq('NP_001317366.1'))

Q06124


In [1]:
import pandas as pd
merge_list=['/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_6_6000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_124419_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_210795_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_256372_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_376154_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_516413_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_553956_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_679658_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_703651_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_778562_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_957145_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_958540_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1109909_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1136434_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1144627_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1177285_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1198762_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1216674_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1253131_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1265114_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1305736_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1634438_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1643299_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1842575_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1869439_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1882377_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1921660_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1939895_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_1960540_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_2005044_0_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_2064664_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_2718339_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_2816690_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_2914851_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_3033508_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_3221904_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_3412166_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_3610615_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_3781720_27000_finished.csv',
'/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/variant_summary_2023/variant_proc_4323149_0_finished.csv']

df=[]
for i,f in enumerate(merge_list):
    f=pd.read_csv(f,index_col='#AlleleID')
    df.append(f)

df_total=pd.concat(df)

In [2]:
df_total.to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/scripts/2023-02-variant-sum.csv')

In [7]:
df_remain=df_total[df_total['UniPort']=='0']

In [10]:
len(df_total)

853517

In [8]:
df_remain.to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/scripts/2023-02-variant-sum_remain.csv')

In [6]:
for idx in df_total.index:
    if 'UniProt' in df_total.loc[idx,'OtherIDs'] and df_total.loc[idx,'UniPort']=='0':
        df_total.loc[idx,'UniPort']=extract_uniprot(df_total.loc[idx,'OtherIDs'])

ClinGen:CA284849,UniProtKB/Swiss-Prot:VAR_003185,OMIM:602671.0001
ClinGen:CA219309,UniProtKB/Swiss-Prot:VAR_007850,OMIM:602671.0003
ClinGen:CA219358,UniProtKB/Swiss-Prot:VAR_016840,OMIM:602671.0013
ClinGen:CA219275,UniProtKB/Swiss-Prot:VAR_025601,OMIM:602671.0015
ClinGen:CA254415,UniProtKB/Swiss-Prot:VAR_001491,OMIM:601011.0001
ClinGen:CA119650,UniProtKB/Swiss-Prot:VAR_001492,OMIM:601011.0002
ClinGen:CA254421,UniProtKB/Swiss-Prot:VAR_001494,OMIM:601011.0004
ClinGen:CA254430,UniProtKB/Swiss-Prot:VAR_043837,OMIM:601011.0011
ClinGen:CA254437,UniProtKB/Swiss-Prot:VAR_043825,OMIM:601011.0009
ClinGen:CA254443,UniProtKB/Swiss-Prot:VAR_043835,OMIM:601011.0012
ClinGen:CA254473,UniProtKB/Swiss-Prot:VAR_043821,OMIM:601011.0017
ClinGen:CA119653,UniProtKB/Swiss-Prot:VAR_043826,OMIM:601011.0018
ClinGen:CA254496,UniProtKB/Swiss-Prot:VAR_043830,OMIM:601011.0027
ClinGen:CA256584,UniProtKB/Swiss-Prot:VAR_010111,OMIM:182389.0001
ClinGen:CA256587,UniProtKB/Swiss-Prot:VAR_010110,OMIM:182389.0002
ClinGen:CA

In [5]:
from tqdm import tqdm
df_remain=df_total[df_total['UniPort']=='0']
pbar = tqdm(total=len(df_remain))

for j,idx in enumerate(df_remain.index):
    df_remain.loc[idx,'UniPort']=get_uniprot_from_name(df_remain.loc[idx,'Name'])
    # print ('\n\n\n\n %d completed \n\n\n\n'%j)
    pbar.update(1)


  0%|          | 1/18409 [00:00<2:50:09,  1.80it/s]

Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001164277.2


  0%|          | 2/18409 [00:01<2:50:08,  1.80it/s]

Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001164277.2


  0%|          | 3/18409 [00:01<2:49:44,  1.81it/s]

Error found in https://rest.uniprot.org/uniprotkb/search?query=NM_001164277.2


  0%|          | 80/18409 [01:17<5:33:14,  1.09s/it]

Error Finding Refseq


  0%|          | 91/18409 [01:29<5:00:44,  1.02it/s]